# 26/04/2023

## Introducción

El objetivo de este trabajo es obtener datos de un activo financiero utilizando técnicas de web scraping y almacenarlos en una base de datos SQL. Estas tareas se realizarán de forma frecuente cada minuto hasta que se decida interrumpir el proceso. Además, se realizará un análisis de los datos obtenidos.

En este caso, se ha seleccionado el activo financiero YPF S.A de Buenos Aires, con el símbolo YPFD en investing.com.

Las librerías que se utilizarán en este proyecto son las siguientes:

* Selenium, para realizar el web scraping.
* sqlite3, para establecer la conexión con la base de datos SQL.
* datetime y schedule, para programar la ejecución repetitiva de la tarea cada 1 minuto.
* csv, para guardar los datos en un archivo CSV.

### Importar las librerías

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import sqlite3
from datetime import datetime
import schedule
import time
import csv


## Web Scraping con Selenium
### Conexión con el driver

In [5]:
# Opciones de navegacion

options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = Service("C:\\Users\\Tatu\\Downloads\\chromedriver_win32\\chromedriver.exe")

driver = webdriver.Chrome(service = driver_path, options = options)


In [6]:
# Conexion con la pagina
driver.get('https://es.investing.com/equities/ypf-sociedad')

tit=driver.title
print(tit)


Acciones YPF | Cotización BA:YPFD hoy - Investing.com


In [4]:
# Insertar fecha y hora actual
now = datetime.now()
print(now)

2023-05-08 14:36:25.905009


### Obtención de los datos
Se debe actualizar el fullXPath, no se mantiene siempre igual

In [ ]:
# Obtencion de los datos con el fullXPath

ultimo_cierre = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[1]/dd/span/span[1]')
ultimo_cierre = ultimo_cierre.text
print("Último cierre " + ultimo_cierre)

rango_min = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[3]/dd/span[1]/span[1]')
rango_min = rango_min.text
print("Mínimo " + rango_min)

rango_max = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[3]/dd/span[3]/span[1]')
rango_max = rango_max.text
print("Máximo " + rango_max)

ingresos_T = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[2]/div[2]/dd/span/span[1]')
ingresos_T = ingresos_T.text
print("Ingresos " + ingresos_T + "T")

apertura = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[2]/dd/span/span[1]')
apertura = apertura.text
print("Apertura " + apertura)


volumen = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[5]/dd/span/span[1]')
volumen = volumen.text
print("Volumen " + volumen)

volumen_promedio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[6]/dd/span/span[1]')
volumen_promedio = volumen_promedio.text
print("Volumen promedio " + volumen_promedio)

ultimo_precio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[1]/div[3]/div/div[1]/div[1]')
ultimo_precio = ultimo_precio.text
print("Último precio " + ultimo_precio)

In [7]:
# Cerrar el driver
driver.close()

## Guardar los datos obtenidos en una base de datos SQL
### Conexión con sqlite3

In [3]:
#Conexión con la base de datos
conn = sqlite3.connect('acciones_YPFD.db')
c = conn.cursor()

In [ ]:
#Creación de la tabla "resumen"
#c.execute('''CREATE TABLE resumen
#             (date_time DATETIME, ultimo_precio REAL, minimo REAL, maximo REAL, ingresos REAL, apertura REAL, volumen REAL, volumen_promedio REAL)''')

In [9]:
# Insertar datos en la tabla
c.execute("INSERT INTO resumen (date_time, ultimo_cierre, minimo, maximo, ingresos, apertura, volumen, volumen_promedio, ultimo_precio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
          (now, ultimo_cierre, rango_min, rango_max, ingresos_T, apertura, volumen, volumen_promedio, ultimo_precio))

# Guardar los cambios en la base de datos y cerrar la conexión
conn.commit()


In [10]:
# Ver los datos guardados
# Seleccionar los datos de la tabla
c.execute("SELECT * FROM resumen")

# Obtener los resultados de la consulta
resultados = c.fetchall()

# Imprimir los resultados en la consola
for fila in resultados:
    print(fila)

('2023-05-05 23:11:14.474165', '4.768,75', 4.841, '5.229,9', '2,53', 4.841, 322.677, 304.817)


In [8]:
# Cerrar la conexión a la base de datos
conn.close()

## Obtención de los datos cada minuto
Utilizamos la librería shedule para que el script se corra cada un minuto e ir obteniendo los valores actualizados. 

Primero creamos la función web_scraping() y copiamos el script utilizado. Esto nos servirá para posteriormente ejecutarlo cada un minuto con schedule.

In [3]:
#Crear la función para utilizar en schedule
def web_scraping():

    #Opciones de navegacion
    options =  webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')

    driver_path = Service("C:\\Users\\Tatu\\Downloads\\chromedriver_win32\\chromedriver.exe")

    driver = webdriver.Chrome(service = driver_path, options = options)

    # Conexion con la pagina
    driver.get('https://es.investing.com/equities/ypf-sociedad')

    # Dejar que cargue la página
    time.sleep(5)

    tit=driver.title
    print(tit)

    # Insertar fecha y hora actual
    now = datetime.now()
    print(now)

    # Obtener los datos con el fullXPath

    ultimo_cierre = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[1]/dd/span/span[1]')
    ultimo_cierre = ultimo_cierre.text
    print("Último cierre " + ultimo_cierre)

    rango_min = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[2]/dd/span[1]/span[1]')
    rango_min = rango_min.text
    print("Mínimo " + rango_min)

    rango_max = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[2]/dd/span[3]/span[1]')
    rango_max = rango_max.text
    print("Máximo " + rango_max)

    ingresos_T = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[3]/dd/span/span[1]')
    ingresos_T = ingresos_T.text
    print("Ingresos " + ingresos_T + "T")

    apertura = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[4]/dd/span/span[1]')
    apertura = apertura.text
    print("Apertura " + apertura)


    volumen = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[7]/dd/span/span[1]')
    volumen = volumen.text
    print("Volumen " + volumen)

    volumen_promedio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/div[2]/main/div/div[8]/div/div[2]/dl/div[10]/dd/span/span[1]')
    volumen_promedio = volumen_promedio.text
    print("Volumen promedio " + volumen_promedio)   

    # Cerrar el driver
    driver.close()

    # Conexion con la base de datos
    conn = sqlite3.connect('acciones_YPFD.db')
    cursor = conn.cursor()

    # Insertar datos en la tabla
    cursor.execute("INSERT INTO resumen (date_time, ultimo_cierre, minimo, maximo, ingresos, apertura, volumen, volumen_promedio, ultimo_precio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
               (now, ultimo_cierre, rango_min, rango_max, ingresos_T, apertura, volumen, volumen_promedio, ultimo_precio))

    # Guardar los cambios en la base de datos y cerrar la conexión
    conn.commit()

    # Ver los datos guardados
    # Seleccionar los datos de la tabla
    cursor.execute("SELECT * FROM resumen")

    # Obtener los resultados de la consulta
    resultados = cursor.fetchall()

    # Imprimir los resultados en la consola
    for fila in resultados:
        print(fila)

    # Cerrar la conexión a la base de datos
    conn.close()



In [6]:
# Ver los datos guardados
# Conexion con la base de datos
conn = sqlite3.connect('acciones_YPFD.db')
cursor = conn.cursor()

# Seleccionar los datos de la tabla
cursor.execute("SELECT * FROM resumen")

# Obtener los resultados de la consulta
resultados = cursor.fetchall()

# Imprimir los resultados en la consola
for fila in resultados:
    print(fila)

# Cerrar la conexión a la base de datos
conn.close()

('2023-05-05 23:11:14.474165', '4.768,75', 4.841, '5.229,9', '2,53', 4.841, 322.677, 304.817)
('2023-05-08 15:10:10.264048', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 158.556, 304.817)
('2023-05-08 15:10:27.563116', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 158.626, 304.817)
('2023-05-08 16:26:44.078323', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 200.111, 304.817)
('2023-05-08 16:27:57.916936', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 202.436, 304.817)
('2023-05-08 16:32:20.396239', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 203.411, 304.817)
('2023-05-08 16:49:32.928998', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 211.215, 304.817)
('2023-05-08 19:31:30.464894', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 229.0, 310.936)
('2023-05-08 19:32:30.624894', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 229.0, 310.936)
('2023-05-08 19:32:54.260681', '5.202,55', 5.217, '5.376,1', '2,53', 5.217, 229.0, 310.936)
('2023-05-08 19:33:47.194170', '5.202,55', 5.217, '5.376,1', '2,53

### Programar la ejecución cada minuto


In [ ]:
# Programar la ejecución del scraping cada minuto
schedule.every().minute.do(web_scraping)

# Ejecutar el programa continuamente
while True:
    schedule.run_pending()
    time.sleep(1)

## Guardar datos en archivo CSV

In [5]:
# Abrir conexión a la base de datos sqlite
conn = sqlite3.connect('acciones_YPFD.db')

# Seleccionar los datos
cursor = conn.cursor()
cursor.execute("SELECT * FROM resumen")

# Crear archivo CSV y escribir datos
with open('resumen_YPF.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    # Escribir encabezados de columnas
    csvwriter.writerow([i[0] for i in cursor.description])  
    csvwriter.writerows(cursor)

# Cerrar conexión a la base de datos
conn.close()


## Conclusión

Se pudieron obtener los datos utilizando el xpath, ya que tanto el nombre como la clase de las variables eran identicos en la mayoría de los casos. Sin embargo, al volver a acceder a la página, el xpath cambió con frecuencia, lo que requirió actualizar constantemente el código. Esto impidió ejecutar la función programada cada minuto como se había planificado utilizando la función de schedule.

Después de ejecutar el script varias veces, enfrentando errores debido a los cambios en el xpath, se lograron obtener 35 valores, como se muestra en la tabla resultante.

Debido a la ineficacia del script para obtener datos de manera frecuente y confiable, se ha decidido continuar el análisis del activo financiero utilizando datos obtenidos a través de una API. Esto permite obtener una mayor cantidad de datos en menos tiempo.

Es importante destacar que una técnica alternativa para realizar web scraping en nuestra URL es el scraping visual. En este enfoque, se realizarían capturas de pantalla de la página web y se utilizaría procesamiento de imágenes para identificar los elementos deseados. Sin embargo, por el momento se descarta esta técnica debido a su mayor complejidad y requerimiento de más tiempo y recursos

